<a href="https://colab.research.google.com/github/Moe-phantom/leetcoding/blob/main/NeuroSymbolicDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm
import datasets

In [14]:
MODEL_NAME = "microsoft/deberta-v3-base"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
MAX_LEN = 128
LR = 2e-5
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 3
BINARY = True

In [15]:
SUSPICIOUS_KEYWORDS = ['allegedly', 'unverified', 'rumor', 'claims', 'exclusive', 'leaked',
                       'breaking', "you won't believe", 'must watch', "doctors hate", 'banned by']
CREDIBLE_KEYWORDS = ['according to', 'reuters', 'bbc', 'associated press', 'verified', 'confirmed', 'peer-reviewed', 'investigation']

def simple_logic_score(text):
    if not isinstance(text, str): return 0.5
    t = text.lower()
    suspicious_count = sum(t.count(k) for k in SUSPICIOUS_KEYWORDS)
    credible_count = sum(t.count(k) for k in CREDIBLE_KEYWORDS)
    words = t.split()
    word_len = max(len(words), 1)
    caps_ratio = sum(1 for w in words if w.isupper() and len(w) > 2) / word_len
    exclam_ratio = t.count('!') / word_len
    score = 0.5 - suspicious_count*0.12 + credible_count*0.12 - caps_ratio*0.15 - exclam_ratio*0.08
    return float(max(0.0, min(1.0, score)))

In [16]:
class LiarDataset(Dataset):
    def __init__(self, df, tokenizer, scaler=None, max_len=128):
        self.texts = df["text"].astype(str).tolist()

        # Dynamic Speaker Features
        self.speaker_cols = [c for c in df.columns if c.startswith("sp_feat_")]
        if not self.speaker_cols:
             self.speakers = np.zeros((len(df), 1), dtype=np.float32)
        else:
             self.speakers = df[self.speaker_cols].values.astype(np.float32)

        self.labels = df["label"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.scaler = scaler

        if self.scaler is not None:
            self.speakers = self.scaler.transform(self.speakers)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        txt = self.texts[idx]
        enc = self.tokenizer(txt, truncation=True, padding='max_length', max_length=self.max_len, return_tensors="pt")
        item = {k: v.squeeze(0) for k,v in enc.items()}
        item['speaker_feats'] = torch.tensor(self.speakers[idx], dtype=torch.float32)

        if BINARY:
            item['label'] = torch.tensor(self.labels[idx], dtype=torch.float32)
        else:
             item['label'] = torch.tensor(self.labels[idx], dtype=torch.long)

        item['logic_score'] = torch.tensor(simple_logic_score(txt), dtype=torch.float32)
        return item

In [17]:
class NeuroSymbolicModel(nn.Module):
    def __init__(self, base_model_name, speaker_feat_dim, hidden_dim=256, binary=True):
        super().__init__()
        self.base = AutoModel.from_pretrained(base_model_name)
        hidden_size = self.base.config.hidden_size

        # Content Head
        self.content_head = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.ReLU()
        )

        # Social Head
        self.social_head = nn.Sequential(
            nn.Linear(speaker_feat_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Logic Head
        self.logic_embed = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU()
        )

        # Fusion (128 + 32 + 16 = 176)
        fusion_dim = 128 + 32 + 16
        self.fusion = nn.Sequential(
            nn.Linear(fusion_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1 if binary else 6)
        )
        self.binary = binary

    def forward(self, input_ids, attention_mask, speaker_feats, logic_score):
        base_out = self.base(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        cls = base_out.last_hidden_state[:,0,:]

        content_feat = self.content_head(cls)
        social_feat = self.social_head(speaker_feats)
        logic_feat = self.logic_embed(logic_score.unsqueeze(1))

        combined = torch.cat([content_feat, social_feat, logic_feat], dim=1)
        logits = self.fusion(combined)

        if self.binary:
            return logits.squeeze(1)
        return logits

In [18]:
def train_epoch(model, loader, optimizer, scheduler, loss_fn):
    model.train()
    losses, preds, trues = [], [], []

    for batch in tqdm(loader, desc="Train"):
        input_ids = batch['input_ids'].to(DEVICE)
        attn = batch['attention_mask'].to(DEVICE)
        sp = batch['speaker_feats'].to(DEVICE)
        logic = batch['logic_score'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        optimizer.zero_grad()
        logits = model(input_ids, attn, sp, logic)

        if BINARY:
            loss = loss_fn(logits, labels)
            prob = torch.sigmoid(logits).detach().cpu().numpy()
            pred = (prob >= 0.5).astype(int)
            true = labels.detach().cpu().numpy().astype(int)
        else:
            loss = loss_fn(logits, labels)
            pred = logits.argmax(dim=1).detach().cpu().numpy()
            true = labels.detach().cpu().numpy()

        loss.backward()
        optimizer.step()
        if scheduler: scheduler.step()

        losses.append(loss.item())
        preds.extend(pred.tolist())
        trues.extend(true.tolist())

    return np.mean(losses), accuracy_score(trues, preds), f1_score(trues, preds, average='binary' if BINARY else 'macro')

In [19]:
def eval_epoch(model, loader, loss_fn):
    model.eval()
    losses, preds, trues = [], [], []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Eval"):
            input_ids = batch['input_ids'].to(DEVICE)
            attn = batch['attention_mask'].to(DEVICE)
            sp = batch['speaker_feats'].to(DEVICE)
            logic = batch['logic_score'].to(DEVICE)
            labels = batch['label'].to(DEVICE)

            logits = model(input_ids, attn, sp, logic)
            if BINARY:
                loss = loss_fn(logits, labels)
                prob = torch.sigmoid(logits).detach().cpu().numpy()
                pred = (prob >= 0.5).astype(int)
                true = labels.detach().cpu().numpy().astype(int)
            else:
                loss = loss_fn(logits, labels)
                pred = logits.argmax(dim=1).detach().cpu().numpy()
                true = labels.detach().cpu().numpy()

            losses.append(loss.item())
            preds.extend(pred.tolist())
            trues.extend(true.tolist())

    return np.mean(losses), accuracy_score(trues, preds), f1_score(trues, preds, average='binary' if BINARY else 'macro')

In [20]:
def hf_to_dataframe(hf_dataset):
    """
    Converts your requested 'chengxuphd/liar2' dataset format
    into a DataFrame compatible with the Neuro-Symbolic model.
    """
    data = []
    print(f"Converting HF Dataset ({len(hf_dataset)} rows)...")

    for item in tqdm(hf_dataset):
        # 1. Text (Combine statement + justification)
        text = str(item['statement'])
        if item.get('justification'):
            text += f" [SEP] {item['justification']}"

        # 2. Label Mapping
        # HF uses ints: 0=false, 1=half-true, 2=mostly-true, 3=true, 4=barely-true, 5=pants-fire
        # We map {1, 2, 3} -> Real (1), others -> Fake (0)
        raw_label = item['label']
        label = 1 if raw_label in [1, 2, 3] else 0

        # 3. Speaker Metadata (The Neuro-Symbolic Requirement)
        try:
            counts = [
                item.get('barely_true_counts', 0),
                item.get('false_counts', 0),
                item.get('half_true_counts', 0),
                item.get('mostly_true_counts', 0),
                item.get('pants_on_fire_counts', 0)
            ]
            # Clean None values
            counts = [float(c) if c is not None else 0.0 for c in counts]

            # Feature Eng
            total = sum(counts) + 1e-5
            credibility = (counts[2] + counts[3]) / total

            # Create feature vector
            sp_feats = [credibility, total] + counts
        except:
            sp_feats = [0.0] * 7 # Fallback

        # Construct Row
        row = {'text': text, 'label': label}
        for i, val in enumerate(sp_feats):
            row[f'sp_feat_{i}'] = val
        data.append(row)

    return pd.DataFrame(data)

In [ ]:
def main():
    print(f"Loading tokenizer: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # ======================================================
    # YOUR REQUESTED DATA LOADING METHOD
    # ======================================================
    print("Downloading chengxuphd/liar2 via datasets...")
    dataset = datasets.load_dataset("chengxuphd/liar2")

    # We convert the splits into DataFrames with metadata attached
    df_train = hf_to_dataframe(dataset["train"])
    df_val = hf_to_dataframe(dataset["validation"])
    df_test = hf_to_dataframe(dataset["test"])

    # Determine speaker dimension
    speaker_cols = [c for c in df_train.columns if c.startswith("sp_feat_")]
    speaker_dim = len(speaker_cols)
    print(f"Detected {speaker_dim} speaker features.")

    # Fit scaler
    print("Fitting scaler...")
    scaler = StandardScaler()
    scaler.fit(df_train[speaker_cols].values.astype(np.float32))

    # Create Datasets
    train_ds = LiarDataset(df_train, tokenizer, scaler=scaler, max_len=MAX_LEN)
    val_ds = LiarDataset(df_val, tokenizer, scaler=scaler, max_len=MAX_LEN)
    test_ds = LiarDataset(df_test, tokenizer, scaler=scaler, max_len=MAX_LEN)

    # Loaders
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Initialize Model
    print(f"Initializing NeuroSymbolicModel on {DEVICE}...")
    model = NeuroSymbolicModel(MODEL_NAME, speaker_feat_dim=speaker_dim, binary=BINARY)
    model.to(DEVICE)

    # Optimizer
    loss_fn = nn.BCEWithLogitsLoss() if BINARY else nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR, weight_decay=WEIGHT_DECAY)
    total_steps = len(train_loader) * NUM_EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.06*total_steps), num_training_steps=total_steps)

    # Training Loop
    best_val_f1 = -1.0
    for epoch in range(NUM_EPOCHS):
        print(f"\n=== Epoch {epoch+1}/{NUM_EPOCHS} ===")
        train_loss, train_acc, train_f1 = train_epoch(model, train_loader, optimizer, scheduler, loss_fn)
        print(f"Train loss: {train_loss:.4f} acc: {train_acc:.4f} f1: {train_f1:.4f}")

        val_loss, val_acc, val_f1 = eval_epoch(model, val_loader, loss_fn)
        print(f"Val   loss: {val_loss:.4f} acc: {val_acc:.4f} f1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_neuro_symbolic.pt")
            print(f"New best model saved! (F1: {val_f1:.4f})")

    # Final Test
    print("\nRunning Final Test...")
    model.load_state_dict(torch.load("best_neuro_symbolic.pt"))
    test_loss, test_acc, test_f1 = eval_epoch(model, test_loader, loss_fn)
    print(f"TEST RESULTS -> Loss: {test_loss:.4f} | Acc: {test_acc:.4f} | F1: {test_f1:.4f}")

In [ ]:
if __name__ == "__main__":
    main()

Loading tokenizer: microsoft/deberta-v3-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

Train:   0%|          | 0/1149 [00:00<?, ?it/s]

Train loss: 0.4036 acc: 0.8225 f1: 0.8604


Eval:   0%|          | 0/144 [00:00<?, ?it/s]

Val   loss: 0.3964 acc: 0.8224 f1: 0.8642
New best model saved! (F1: 0.8642)

=== Epoch 3/3 ===


Train:   0%|          | 0/1149 [00:00<?, ?it/s]

usage: colab_kernel_launcher.py [-h] --train TRAIN --val VAL [--test TEST]
colab_kernel_launcher.py: error: the following arguments are required: --train, --val
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/argparse.py", line 1943, in _parse_known_args2
    namespace, args = self._parse_known_args(args, namespace, intermixed)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2230, in _parse_known_args
    raise ArgumentError(None, _('the following arguments are required: %s') %
argparse.ArgumentError: the following arguments are required: --train, --val

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-700794115.py", line 9, in <cell line: 0>
    args = parser.parse_args()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 1904, in parse_args
    args, argv = self.parse_known_args(args, namesp

TypeError: object of type 'NoneType' has no len()